<a href="https://colab.research.google.com/github/beiyouwuyanzu/llama_application/blob/main/Copy_of_Chinese_Vicuna_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown "https://drive.google.com/uc?id=1tzXVhS74m-EtoFot7hEc005LDeZGPit_" -O merge.json

Downloading...
From: https://drive.google.com/uc?id=1tzXVhS74m-EtoFot7hEc005LDeZGPit_
To: /content/merge.json
100% 696M/696M [00:03<00:00, 195MB/s]


In [ ]:
!git clone https://github.com/Facico/Chinese-Vicuna
!pip install -r ./Chinese-Vicuna/requirements.txt

In [ ]:
!python ./Chinese-Vicuna/finetune.py --data_path merge.json --test_size 2000

In [ ]:
import os
import sys

import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
import argparse
import warnings


assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
)

In [4]:
DATA_PATH = "./merge.json" #"/home/cciip/private/fanchenghao/dataset/instruction/merge.json"
OUTPUT_DIR = "./output" #"lora-Vicuna"

device_map = "auto"
world_size = int(os.environ.get("WORLD_SIZE", 1))

In [ ]:
model_path = "decapoda-research/llama-7b-hf"

ddp = world_size != 1
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size
print(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=True,
    device_map=device_map,
)

In [8]:
tokenizer = LlamaTokenizer.from_pretrained(
    model_path, add_eos_token=True
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [11]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
model = get_peft_model(model, config)

In [13]:
tokenizer.pad_token_id = 0

In [14]:
VAL_SET_SIZEdata = load_dataset("json", data_files=DATA_PATH)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9a2b36bbad088d0b/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
VAL_SET_SIZE = 2000
MICRO_BATCH_SIZE = 4  # this could actually be 5 but i like powers of 2
BATCH_SIZE = 128
EPOCHS = 3  # we don't always need 3 tbh
now_max_steps = max((len(data["train"]) - VAL_SET_SIZE) // BATCH_SIZE * EPOCHS, EPOCHS)

In [19]:
print(now_max_steps)

16218


In [20]:
MAX_STEPS = now_max_steps

In [21]:
model.print_trainable_parameters()

trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199


In [22]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""

In [23]:
def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }


In [24]:
def generate_and_tokenize_prompt(data_point):
    # This function masks out the labels for the input,
    # so that our loss is computed only on the response.
    user_prompt = (
        (
            f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
"""
        )
        if data_point["input"]
        else (
            f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
"""
        )
    )
    len_user_prompt_tokens = (
        len(
            tokenizer(
                user_prompt,
                truncation=True,
                max_length=CUTOFF_LEN + 1,
            )["input_ids"]
        )
        - 1
    )  # no eos token
    full_tokens = tokenizer(
        user_prompt + data_point["output"],
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )["input_ids"][:-1]
    return {
        "input_ids": full_tokens,
        "labels": [-100] * len_user_prompt_tokens
        + full_tokens[len_user_prompt_tokens:],
        "attention_mask": [1] * (len(full_tokens)),
    }



In [25]:
print(VAL_SET_SIZE)

2000


In [27]:
LEARNING_RATE = 3e-4  # the Karpathy constant
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05


train_val = data["train"].train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"].shuffle().map(generate_and_tokenize_prompt)
val_data = train_val["test"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/691987 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [28]:
print(train_data)

Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'labels', 'attention_mask'],
    num_rows: 691987
})


In [ ]:
val_data[0]

In [53]:
print(val_data[3])

{'instruction': '分析软件中的细节。', 'input': '计算器软件', 'output': '计算器软件涵盖了多种数学算法，可以让用户快速、准确地进行复杂的数学运算。它还包括一些实用功能，如支持多种进制、计算器历史记录，以及可以快速计算常用函数，例如三角函数、对数、指数等。此外，计算器软件还支持四则运算、括号表达式、变量定义和抽象等功能，以及可以显示和编辑公式的功能，方便快捷地进行数学操作。', 'input_ids': [0, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29892, 3300, 2859, 411, 385, 1881, 393, 8128, 4340, 3030, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 30748, 233, 161, 147, 235, 192, 178, 30631, 30275, 30210, 234, 190, 137, 31669, 30267, 13, 13, 2277, 29937, 10567, 29901, 13, 31466, 31565, 30943, 235, 192, 178, 30631, 13, 13, 2277, 29937, 13291, 29901, 13, 31466, 31565, 30943, 235, 192, 178, 30631, 233, 185, 184, 234, 158, 153, 30743, 30923, 31893, 30354, 30415, 31565, 30545, 30214, 30682, 30651, 235, 177, 172, 30406, 31229, 232, 194, 174, 31859, 30330, 232, 138, 137, 31835, 30533, 31174, 30448, 31810, 233, 160, 133, 30210, 30354, 30415, 31894, 31565, 30267, 232, 177, 134, 31994, 31473, 233, 1

In [52]:
print(example[3])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 3

In [32]:
print(example.keys())

dict_keys(['instruction', 'input', 'output', 'input_ids', 'labels', 'attention_mask'])


In [35]:
for k, v in example.items():
  print(k, len(v), v)

instruction 22 给我推荐一些适合夏天穿的款式时尚衣服。\n

input 0 
output 108 1. 连体短裤/连衣裙
2. 露肩上衣/连衣裙
3. 宽松T恤
4. 花卉印花短裙
5. 纯棉针织背心
6. 薄棉质长裤
7. 无袖连衣裙
8. 男/女运动套装
9. 水蓝色或白色短裤/短裙
10. 宽边草帽和太阳镜
input_ids 256 [0, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 31999, 30672, 233, 145, 171, 235, 144, 147, 30287, 31959, 236, 131, 133, 30733, 31241, 30408, 234, 172, 194, 30210, 233, 175, 193, 30607, 30594, 232, 179, 157, 235, 164, 166, 31520, 30267, 29905, 29876, 13, 13, 13, 2277, 29937, 13291, 29901, 13, 29896, 29889, 29871, 31903, 30988, 234, 162, 176, 235, 166, 167, 29914, 31903, 235, 164, 166, 235, 166, 156, 13, 29906, 29889, 29871, 236, 159, 181, 235, 133, 172, 30429, 235, 164, 166, 29914, 31903, 235, 164, 166, 235, 166, 156, 13, 29941, 29889, 29871, 232, 177, 192, 31018, 29911, 233, 132, 167, 13, 29946, 29889, 29871, 30830, 232, 144, 140, 232, 144, 179, 30830, 234, 162, 176, 235, 166, 156, 13, 2994

# 模型预测

In [48]:
def generate_predict_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""


prompt = generate_predict_prompt("中国哪个省份面积比较大")
inputs = tokenizer(prompt, return_tensors="pt")

In [49]:
print(inputs)

{'input_ids': tensor([[    0, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29889,
         14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
         29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13, 30275,
         30356,   232,   150,   173, 30502, 31600,   231,   190,   192, 30806,
           234,   170,   178, 31419,   235,   193,   134, 30257,    13,    13,
          2277, 29937, 13291, 29901,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}


In [50]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
temperature=0.1
top_p=0.75
top_k=40
num_beams=4
max_new_tokens=128
min_new_tokens=1
repetition_penalty=2.0

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

input_ids = inputs["input_ids"].to(device)

generation_config = GenerationConfig(
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    num_beams=num_beams,
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    max_new_tokens=max_new_tokens, # max_length=max_new_tokens+input_sequence
    min_new_tokens=min_new_tokens, # min_length=min_new_tokens+input_sequence
    # **kwargs,
)

In [54]:
with torch.no_grad():
    generation_output = model.generate(
      input_ids=input_ids,
      generation_config=generation_config,
      return_dict_in_generate=True,
      output_scores=False,
      repetition_penalty=1.3,
    )
    output = generation_output.sequences[0]
    output = tokenizer.decode(output).split("### Response:")[1].strip()
    print(output)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<unk>
\begin{itemize}
\item 中国北方面积比较大
\item 中国东方面积比较大
\item 中国西方面积比较大
\item 中国南方面积比较大
\item 中国北方面积比较小
\item 中国东方面积比较小
\item 中国西方面积比较小
\item


In [56]:
# 处理训练数据

In [57]:
!unzip Math_data.zip

Archive:  Math_data.zip
   creating: Math_data/
  inflating: __MACOSX/._Math_data    
  inflating: Math_data/math23k_train.json  
  inflating: __MACOSX/Math_data/._math23k_train.json  
  inflating: Math_data/Math23K.docx  
  inflating: __MACOSX/Math_data/._Math23K.docx  
  inflating: Math_data/License.pdf   
  inflating: __MACOSX/Math_data/._License.pdf  
  inflating: Math_data/math23k_test.json  
  inflating: __MACOSX/Math_data/._math23k_test.json  


In [59]:
! wc -l /content/Math_data/math23k_train.json

155134 /content/Math_data/math23k_train.json


In [62]:
! head -n 20 /content/Math_data/math23k_train.json

{
    "id":"1",
    "original_text":"镇海雅乐学校二年级的小朋友到一条小路的一边植树．小朋友们每隔2米种一棵树（马路两头都种了树），最后发现一共种了11棵，这条小路长多少米．",
    "segmented_text":"镇海 雅乐 学校 二年级 的 小朋友 到 一条 小路 的 一边 植树 ． 小朋友 们 每隔 2 米 种 一棵树 （ 马路 两头 都 种 了 树 ） ， 最后 发现 一共 种 了 11 棵 ， 这 条 小路 长 多少 米 ．",
    "equation":"x=(11-1)*2",
    "ans":"20"
}
{
    "id":"2",
    "original_text":"一个工程队挖土，第一天挖了316方，从第二天开始每天都挖230方，连续挖了6天，这个工程队一周共挖土多少方？",
    "segmented_text":"一 个 工程队 挖土 ， 第一天 挖 了 316 方 ， 从 第 二 天 开始 每天 都 挖 230 方 ， 连续 挖 了 6 天 ， 这个 工程队 一周 共 挖土 多少 方 ？",
    "equation":"x=316+230*(6-1)",
    "ans":"1466"
}
{
    "id":"3",
    "original_text":"小明看一本故事书，第一天看了全书的(1/6)，第二天看了24页，第3天看的页数是前两天看的总数的150%，这时还有全书的(1/4)没有看，那么这本书一共多少页．",
    "segmented_text":"小 明 看 一本 故事书 ， 第一天 看 了 全书 的 (1/6) ， 第 二 天 看 了 24 页 ， 第 3 天 看 的 页数 是 前两天 看 的 总数 的 150% ， 这时 还有 全书 的 (1/4) 没有 看 ， 那么 这 本书 一共 多少 页 ．",
    "equation":"x=(24+24*150%)/(1-(1/6)-(1/6)*150%-(1/4))",
    "ans":"180"


In [68]:
import json

# 加载数据
def make_train_data(fi, ou):
  with open(ou, 'w') as w:
    w.write("[")
    cache = ""
    with open(fi) as f:
      for line in f:
        cache += line.strip()
        if line.startswith("}"):
          
          # print(cache)
          js = json.loads(cache)
          data = {}
          data['instruction'] = js['original_text']
          data['input'] = ''
          data['output'] = js['equation'] + '答案等于' + js['ans']
          # print(data)
          w.write(json.dumps(data))

          cache = ""
    w.write(']')


make_train_data('/content/Math_data/math23k_train.json', 'train_input')


    

# train_ori_data = load_dataset("json", data_files="/content/Math_data/math23k_train.json")

In [69]:
make_train_data('/content/Math_data/math23k_test.json', 'test_input')

In [70]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [74]:
! ls /content/drive/MyDrive/llama_data

test_input  train_input
